# In this notebook we will read in the original ROOT files and convert them to other formats more suitable for ML.
## We will apply a common preselection and split the data into barrel and endcap dataframes, which we will store separately.

### First standard imports

In [2]:
%matplotlib inline
import IPython


### Define location and name of the input ROOT file

In [3]:
filename = '/eos/cms/store/group/phys_egamma/ReleaseInputsArchive/2017UL_ElePhoReg/input_trees/DoubleElectron_FlatPt-1To300_2017ConditionsFlatPU0to70ECALGT_105X_mc2017_realistic_IdealEcalIC_v5-v2_AODSIM_EgRegTreeV5Refined.root'

### Imports to be able to read ROOT, and output 'parquet' and 'feather' files, which are top2 formats for ML

In [4]:
from root_pandas import read_root
from progressbar import ProgressBar
import pandas as pd
import fastparquet
import pyarrow.feather as feather


Welcome to JupyROOT 6.18/00


### Start reading and storing one million events per time. Both formats are used as outputs. One can switch to just one, but extra work is not terribly huge.

In [5]:
%time
# Grab a coffee ... and a donnut! It will take 34 mins with 4 cores and 10/16 gb of memory setup
pbar = ProgressBar()
count = 1
oneMillion=1000000
for df in pbar(read_root(filename, chunksize=oneMillion)):
    print ('count = ', count)
    feather.write_feather(df, 'df_{0}.feather'.format(count))
    df.to_parquet('df_{0}.parquet'.format(count), engine='fastparquet', compression='gzip')
    count +=1

CPU times: user 0 ns, sys: 11 µs, total: 11 µs
Wall time: 22.2 µs


N/A% (0 of 20) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

('count = ', 1)


  5% (1 of 20) |#                        | Elapsed Time: 0:01:48 ETA:   0:34:27

('count = ', 2)


...

 90% (18 of 20) |#####################   | Elapsed Time: 0:32:30 ETA:   0:03:38

('count = ', 19)


 95% (19 of 20) |######################  | Elapsed Time: 0:33:50 ETA:   0:01:19

('count = ', 20)


100% (20 of 20) |########################| Elapsed Time: 0:34:15 Time:  0:34:15


### Look at the columns in the dataframe. Notice, this dataframe is the LAST chunck read from ROOT file and contains a million events or less. 

In [12]:
df.dtypes

nrVert                                int32
rho                                 float32
nrPUInt                             float32
nrPUIntTrue                         float32
evt_runnr                             int32
evt_lumiSec                           int32
evt_eventnr                           int32
sc_rawEnergy                        float32
sc_rawESEnergy                      float32
sc_etaWidth                         float32
sc_phiWidth                         float32
sc_seedClusEnergy                   float32
sc_numberOfClusters                 float32
sc_numberOfSubClusters              float32
sc_clusterMaxDR                     float32
sc_clusterMaxDRDPhi                 float32
sc_clusterMaxDRDEta                 float32
sc_clusterMaxDRRawEnergy            float32
sc_corrEnergy                       float32
sc_scEta                            float32
sc_scPhi                            float32
sc_seedEta                          float32
sc_seedPhi                      

### Define all possible electron variables of interest. 


In [5]:
ele_vars = [
#=== Full electron object ===
'ele_ecalDrivenSeed',
'ele_et',
'ele_fbrem',
'ele_hademTow',
'ele_nrSatCrys',
'ele_trkEtaMode',
'ele_trkPMode',
'ele_trkPModeErr',
'ele_trkPhiMode',

#=== Possibly: for splitting into subsamples if several regressions are chained ===
'evt_eventnr',

#=== MC truth info ===
'mc_energy',

#=== Global branch, pileup ===
'rho',

#=== SuperCluster variables
'sc_dEtaSeedSC',
'sc_dPhiSeedSC',
'sc_etaWidth',
'sc_iEtaMod20',
'sc_iEtaMod5',
'sc_iEtaOrX',
'sc_iPhiMod2',
'sc_iPhiMod20',
'sc_iPhiOrY',
'sc_isEB',
'sc_numberOfClusters',
'sc_phiWidth',
'sc_rawESEnergy',
'sc_rawEnergy',
'sc_seedClusEnergy',

#=== Shower Shapes that are not full 5x5 "ssFrac" (used for selection only) ====
'ssFrac_sigmaIEtaIEta',
'ssFrac_sigmaIPhiIPhi',

#=== Shower Shapes that are full 5x5 "ssFull" ===
'ssFull_e2nd',
'ssFull_e2x5Bottom',
'ssFull_e2x5Left',
'ssFull_e2x5Max',
'ssFull_e2x5Right',
'ssFull_e2x5Top',
'ssFull_e3x3',
'ssFull_e5x5',
'ssFull_eBottom',
'ssFull_eLeft',
'ssFull_eMax',
'ssFull_eRight',
'ssFull_eTop',
'ssFull_sigmaIEtaIEta',
'ssFull_sigmaIEtaIPhi',
'ssFull_sigmaIPhiIPhi'
]



In [6]:
len(ele_vars)

45

### Sanity check of the dataframe. It is optional! Let's look at the variables and print min/max/std and other good indicators. Subselect only variables defined above.

In [14]:
ndf = df[df.columns & ele_vars]
ndf

,rho,evt_eventnr,sc_rawEnergy,sc_rawESEnergy,sc_etaWidth,sc_phiWidth,sc_seedClusEnergy,sc_numberOfClusters,sc_dEtaSeedSC,sc_dPhiSeedSC,...,ele_et,ele_trkEtaMode,ele_trkPhiMode,ele_trkPMode,ele_trkPModeErr,ele_fbrem,ele_hademTow,ele_ecalDrivenSeed,ele_nrSatCrys,mc_energy
19000000,0.560577,7483004,100.162697,0.000000,0.010192,0.009672,100.162697,1.0,0.000000e+00,-1.110223e-16,...,101.542236,-0.198346,0.854715,108.501755,5.722579,0.101118,0.000000,1.0,0.0,103.414360
19000001,0.560577,7483004,102.297104,0.000000,0.007167,0.007494,102.297104,1.0,0.000000e+00,0.000000e+00,...,101.419838,0.198721,-2.286952,102.516655,5.853707,-0.021650,0.000000,1.0,0.0,103.414360
19000002,0.000000,7483010,815.802429,30.942381,0.014211,0.015846,814.839050,2.0,-6.217502e-05,1.870113e-04,...,237.282791,1.938216,-0.574147,849.159851,245.585541,0.452575,0.001015,1.0,0.0,849.609924
19000003,0.000000,7483010,825.615601,16.534718,0.012099,0.014092,824.827087,2.0,5.456224e-05,-2.009709e-04,...,236.893997,-1.939229,2.567113,1073.643311,359.965485,0.704514,0.000000,1.0,0.0,849.609924
19000004,2.722099,7483003,127.545967,0.000000,0.007443,0.023278,119.252518,3.0,-4.991862e-05,-5.259997e-03,...,128.327942,0.134032,-2.000583,147.272049,17.983248,0.947268,0.000000,1.0,0.0,130.602478
19000005,2.722099,7483003,129.278946,0.000000,0.008793,0.008621,129.278946,1.0,0.000000e+00,0.000000e+00,...,128.911102,-0.133630,1.140620,128.044632,7.525030,0.025384,0.000000,1.0,0.0,130.602478
19000006,3.313263,7483008,119.501480,0.000000,0.008551,0.008378,119.501480,1.0,0.000000e+00,0.000000e+00,...,120.827484,0.141874,1.226968,117.740089,7.987118,0.173061,0.000000,1.0,0.0,123.540627
19000007,3.313263,7483008,118.483543,0.000000,0.009606,0.012668,118.483543,1.0,0.000000e+00,0.000000e+00,...,119.242516,-0.141794,-1.914681,135.964798,20.876345,0.951372,0.000000,1.0,0.0,123.540627
19000008,5.424200,7483013,479.678101,25.706406,0.021173,0.027684,479.678101,1.0,0.000000e+00,0.000000e+00,...,89.590187,-2.441975,2.608145,158.790802,12.439394,-0.100572,0.002859,1.0,0.0,522.065369
19000009,5.424200,7483013,480.297455,24.934132,0.023404,0.024956,480.297455,1.0,0.000000e+00,0.000000e+00,...,89.187332,2.447175,-0.534207,28.350367,1.785440,0.059055,0.000000,1.0,0.0,522.065369


### Now it is time to combine individual 'parquet' or 'feather' files we already stored in the 'grand' dataframe and store that one to disk under the name 'comb_df'.

In [31]:
%time
list_of_dfs = []
numOfFiles = 20

# Here we are picking just one of the formats to read, but still store BOTH.
nameTemplate = 'df_{0}.parquet'
fileLocation = '/eos/user/r/rkamalie/'
print_read_df = False

for i in range(1,numOfFiles+1):
    print ('Processing {0} out of {1} files'.format(i, numOfFiles))

    fileToProcess = fileLocation + nameTemplate.format(i)
    print 'fileToProcess=', fileToProcess

    if 'feather' in nameTemplate:
        read_df = feather.read_feather(fileToProcess)
    elif 'parquet' in nameTemplate:
        read_df = pd.read_parquet(fileToProcess)
    else:
        print 'This should not happen, nameTemplate is wrong, please check it is in parquet or feather format or that the template correctly describes the existing files, \
exiting...'
        sys.exit(1)

    if print_read_df:
        print read_df.info(memory_usage='deep')
        print '-'*50
        print read_df.describe()

    # x4 reduction in size                                                                                                                                                  
    reduced_df = read_df[read_df.columns & ele_vars]

    if print_read_df:
        print reduced_df.info(memory_usage='deep')
        print '-'*50
        print reduced_df.describe()

    list_of_dfs.append(reduced_df)

print 'Start concatenating dataframes, it may take some time, about 5-6 minutes with memory usage of 3.3 GB'
comb_df = pd.concat(list_of_dfs, ignore_index=True)

print '################################################################################'
print '##################### INFO ABOUT COMBINED DATAFRAME ############################'

print comb_df.info(memory_usage='deep')
print '-'*50
print comb_df.describe()

print '################################## END #########################################'
print '################################################################################'

print 'Write this total dataframe to disk'

print 'Processing feather'
feather.write_feather(comb_df, 'comb_df.feather')

print 'Processing parquet'
comb_df.to_parquet('comb_df.parquet', engine='fastparquet', compression='gzip')

print 'All done!'

CPU times: user 21 µs, sys: 1 µs, total: 22 µs
Wall time: 36 µs
Processing 1 out of 20 files
fileToProcess= /eos/user/r/rkamalie/df_1.parquet
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 170 entries, nrVert to clus3_clusterDPhiToSeed
dtypes: float32(166), int32(4)
memory usage: 648.5 MB
None
--------------------------------------------------
               nrVert             rho         nrPUInt     nrPUIntTrue  \
count  1000000.000000  1000000.000000  1000000.000000  1000000.000000   
mean        29.260222       20.199434       35.535400       35.539474   
std         18.083026       12.836236       21.336018       20.488903   
min          1.000000        0.000000        0.000000        0.000056   
25%         15.000000        9.444513       17.000000       17.791363   
50%         28.000000       19.689667       35.000000       35.572479   
75%         42.000000       29.889420       53.000000       53.289472   
max        129.000000       7

In [32]:
comb_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19414000 entries, 0 to 19413999
Data columns (total 45 columns):
rho                     float32
evt_eventnr             int32
sc_rawEnergy            float32
sc_rawESEnergy          float32
sc_etaWidth             float32
sc_phiWidth             float32
sc_seedClusEnergy       float32
sc_numberOfClusters     float32
sc_dEtaSeedSC           float32
sc_dPhiSeedSC           float32
sc_isEB                 float32
sc_iEtaOrX              float32
sc_iPhiOrY              float32
sc_iEtaMod5             float32
sc_iPhiMod2             float32
sc_iEtaMod20            float32
sc_iPhiMod20            float32
ssFull_e3x3             float32
ssFull_e5x5             float32
ssFull_eMax             float32
ssFull_e2nd             float32
ssFull_sigmaIEtaIEta    float32
ssFull_sigmaIEtaIPhi    float32
ssFull_sigmaIPhiIPhi    float32
ssFull_e2x5Max          float32
ssFull_e2x5Top          float32
ssFull_e2x5Bottom       float32
ssFull_e2x5Left        

## Here are the cuts defined by EGamma. We reimplement them for Pandas. 
Regression.1.CutBase: 
(mc.energy>0 && ssFrac.sigmaIEtaIEta>0 && ssFrac.sigmaIPhiIPhi>0 && ele.et>0 && evt.eventnr%10==0)                                               


In [43]:
mydf = comb_df[(comb_df.mc_energy>0) & (comb_df.ssFrac_sigmaIEtaIEta>0) & (comb_df.ssFrac_sigmaIPhiIPhi>0) & (comb_df.ele_et>0)]
mydf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14985168 entries, 0 to 19413999
Data columns (total 45 columns):
rho                     float32
evt_eventnr             int32
sc_rawEnergy            float32
sc_rawESEnergy          float32
sc_etaWidth             float32
sc_phiWidth             float32
sc_seedClusEnergy       float32
sc_numberOfClusters     float32
sc_dEtaSeedSC           float32
sc_dPhiSeedSC           float32
sc_isEB                 float32
sc_iEtaOrX              float32
sc_iPhiOrY              float32
sc_iEtaMod5             float32
sc_iPhiMod2             float32
sc_iEtaMod20            float32
sc_iPhiMod20            float32
ssFull_e3x3             float32
ssFull_e5x5             float32
ssFull_eMax             float32
ssFull_e2nd             float32
ssFull_sigmaIEtaIEta    float32
ssFull_sigmaIEtaIPhi    float32
ssFull_sigmaIPhiIPhi    float32
ssFull_e2x5Max          float32
ssFull_e2x5Top          float32
ssFull_e2x5Bottom       float32
ssFull_e2x5Left        

### Save now preselected 'grand' dataframe.

In [48]:
%time
print 'Processing feather'
feather.write_feather(mydf, 'preselected_comb_df.feather')

print 'Processing parquet'
mydf.to_parquet('preselected_comb_df.parquet', engine='fastparquet', compression='gzip')

print 'All done!'

Processing feather
Processing parquet
All done!


### Define usefull function to measure the RAM taking by the notebook.

In [122]:
#memory usage and release 
#https://stackoverflow.com/questions/39100971/how-do-i-release-memory-used-by-a-pandas-dataframe

import os, psutil, numpy as np
import gc
    
def usage():
    process = psutil.Process(os.getpid())
    return process.memory_info()[0] / float(2 ** 20)

def garbageCollect():
    gc.collect()

In [5]:
from notebook.services.config import ConfigManager
ConfigManager().update('notebook', {'ExecuteTime': {
    'display_absolute_timestamps': False,
    'relative_timing_update_period': 5,
    'template': {
        'executed': 'started ${start_time}, finished in ${duration}',
    }
}})


{u'ExecuteTime': {u'display_absolute_timestamps': False,
  u'relative_timing_update_period': 5,
  u'template': {u'executed': 'started ${start_time}, finished in ${duration}'}},
 u'load_extensions': {u'jupyter-js-widgets/extension': True,
  u'rise/main': True}}

### Some usefull commands for notebooks
https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts


### If anywhere by chance the system dies but preselected dataframe is stored, here is how to read it back for 'parquet':

In [29]:
import pandas as pd
#%timeit 
mydf = pd.read_parquet('preselected_comb_df.parquet')
# takes about 20 seconds


### And here is how to read it for 'feather' format.

In [123]:
import pandas as pd
import pyarrow.feather as feather
#%timeit 
mydf = feather.read_feather('preselected_comb_df.feather')
#takes 5 to 55 seconds
### PREFER feather SINCE IT REINDEXES

In [124]:
usage()

3551.95703125

### If the namespace is full (output of usage() is close to the notebook limit (10-16 Gb depending on setting you chose)), you may want to check what exactly occupies the napespace. For that, use 'whos' command. Then, you can delete intidivual items using 'del nameOfObject' command.

In [48]:
#del mydf

0

In [7]:
usage()

2793.13671875

In [6]:
mydf.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14985168 entries, 0 to 14985167
Data columns (total 45 columns):
rho                     float32
evt_eventnr             int32
sc_rawEnergy            float32
sc_rawESEnergy          float32
sc_etaWidth             float32
sc_phiWidth             float32
sc_seedClusEnergy       float32
sc_numberOfClusters     float32
sc_dEtaSeedSC           float32
sc_dPhiSeedSC           float32
sc_isEB                 float32
sc_iEtaOrX              float32
sc_iPhiOrY              float32
sc_iEtaMod5             float32
sc_iPhiMod2             float32
sc_iEtaMod20            float32
sc_iPhiMod20            float32
ssFull_e3x3             float32
ssFull_e5x5             float32
ssFull_eMax             float32
ssFull_e2nd             float32
ssFull_sigmaIEtaIEta    float32
ssFull_sigmaIEtaIPhi    float32
ssFull_sigmaIPhiIPhi    float32
ssFull_e2x5Max          float32
ssFull_e2x5Top          float32
ssFull_e2x5Bottom       float32
ssFull_e2x5Left        

### Settings to force the dataframe display all columns and not the truncted version.

In [125]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
mydf

,rho,evt_eventnr,sc_rawEnergy,sc_rawESEnergy,sc_etaWidth,sc_phiWidth,sc_seedClusEnergy,sc_numberOfClusters,sc_dEtaSeedSC,sc_dPhiSeedSC,sc_isEB,sc_iEtaOrX,sc_iPhiOrY,sc_iEtaMod5,sc_iPhiMod2,sc_iEtaMod20,sc_iPhiMod20,ssFull_e3x3,ssFull_e5x5,ssFull_eMax,ssFull_e2nd,ssFull_sigmaIEtaIEta,ssFull_sigmaIEtaIPhi,ssFull_sigmaIPhiIPhi,ssFull_e2x5Max,ssFull_e2x5Top,ssFull_e2x5Bottom,ssFull_e2x5Left,ssFull_e2x5Right,ssFull_eTop,ssFull_eBottom,ssFull_eLeft,ssFull_eRight,ssFrac_sigmaIEtaIEta,ssFrac_sigmaIPhiIPhi,ele_et,ele_trkEtaMode,ele_trkPhiMode,ele_trkPMode,ele_trkPModeErr,ele_fbrem,ele_hademTow,ele_ecalDrivenSeed,ele_nrSatCrys,mc_energy
0,10.558399,9136010,96.798744,0.000000,0.006568,0.031675,70.552330,4.0,-5.031547e-04,-2.415152e-03,1.0,32.0,256.0,1.0,1.0,6.0,15.0,70.489532,84.388367,48.542126,9.435896,0.007908,0.065228,0.017998,79.950562,22.733250,7.897033,3.577513,8.699830,9.435896,3.456726,1.023364,3.785786,0.009104,0.013606,86.204430,0.523132,-2.002805,78.652199,7.454437,0.867825,0.000000,1.0,0.0,99.399963
1,10.558399,9136010,97.621788,0.000000,0.009281,0.010019,97.621788,1.0,0.000000e+00,0.000000e+00,1.0,-28.0,75.0,-2.0,0.0,-2.0,14.0,93.421600,98.287766,31.366396,29.315327,0.008893,-0.100080,0.009975,94.656113,52.960716,1.944843,1.965344,33.932362,29.315327,0.594810,0.466266,10.753372,0.008869,0.009995,87.683220,-0.522876,1.139411,97.622147,4.116127,0.316007,0.000000,1.0,0.0,99.399963
2,12.537903,9136005,221.094452,0.000000,0.008480,0.012603,220.320511,2.0,3.056435e-05,-5.005577e-04,1.0,-21.0,101.0,0.0,0.0,0.0,0.0,211.077713,219.042786,167.985031,14.997272,0.009775,0.009249,0.010154,206.873917,16.643719,15.036918,25.694096,9.460797,7.501907,6.679216,14.997272,3.580314,0.009775,0.010154,206.245773,-0.370815,1.573278,236.226303,23.031010,0.301949,0.000000,1.0,0.0,223.497986
3,12.537903,9136005,217.442139,0.000000,0.010157,0.010345,217.442139,1.0,0.000000e+00,0.000000e+00,1.0,22.0,280.0,1.0,1.0,1.0,19.0,206.086761,216.032410,131.924179,36.406002,0.009165,0.084857,0.010268,205.479294,34.932705,9.332662,57.994270,5.643030,16.092178,2.854428,36.406002,1.705848,0.009156,0.010272,209.821930,0.370779,-1.568233,199.691940,16.695520,-0.021369,0.003535,1.0,0.0,223.497986
4,12.807579,9136002,189.944199,0.000000,0.006963,0.030441,181.715485,5.0,6.359640e-04,-1.577192e-04,1.0,-53.0,9.0,-2.0,0.0,-7.0,8.0,166.529587,181.445374,84.001167,60.661690,0.008288,0.046061,0.014327,169.458038,9.336192,79.387581,11.480898,10.961783,4.525568,60.661690,4.541782,3.556031,0.008307,0.014326,131.120331,-0.933664,-0.028918,195.827820,28.702194,0.564550,0.000000,1.0,0.0,193.680542
5,5.446676,9136017,12.729020,0.000000,0.007266,0.006614,12.729020,1.0,0.000000e+00,-1.110223e-16,1.0,2.0,49.0,1.0,0.0,1.0,8.0,13.188349,13.515256,8.817220,1.665064,0.009273,-0.146166,0.009757,13.133986,0.427279,2.535194,2.791915,0.268336,0.139834,1.497952,1.665064,0.070499,0.009270,0.009264,14.025374,0.025887,0.613671,14.110765,0.351522,0.005716,0.000000,1.0,0.0,14.144497
6,5.446676,9136017,12.650763,0.000000,0.006147,0.008270,12.650763,1.0,0.000000e+00,0.000000e+00,1.0,-1.0,223.0,0.0,0.0,0.0,2.0,12.817404,13.760635,7.355963,4.009751,0.010460,0.085742,0.009192,12.891888,0.475759,4.883085,0.599375,1.558402,0.099447,4.009751,0.244135,0.648014,0.009629,0.008696,14.108038,-0.024787,-2.528021,14.463190,0.449630,0.028391,0.000000,1.0,0.0,14.144497
7,17.500393,9136008,107.924393,8.423841,0.012864,0.047712,82.378540,3.0,-2.167862e-03,6.474053e-03,0.0,20.0,27.0,0.0,0.0,0.0,0.0,82.253563,103.202377,51.282570,13.834238,0.021894,-0.061254,0.055772,84.722588,27.398100,8.006230,36.951664,7.822662,4.353574,1.782115,13.834238,2.012559,0.026104,0.032817,42.754639,1.740872,-2.509806,20.668043,2.414800,0.691514,0.000000,1.0,0.0,128.248749
8,17.500393,9136008,108.024261,11.480968,0.014954,0.044356,102.896591,3.0,1.522460e-03,-3.945758e-03,0.0,82.0,74.0,0.0,0.0,0.0,0.0,95.537163,102.174324,44.451820,31.989223,0.022739,0.035947,0.037725,94.638268,8.885093,14.859746,4.069554,45.262173,4.570642,2.954343,1.382553

In [9]:
mydf.describe()

,rho,evt_eventnr,sc_rawEnergy,sc_rawESEnergy,sc_etaWidth,sc_phiWidth,sc_seedClusEnergy,sc_numberOfClusters,sc_dEtaSeedSC,sc_dPhiSeedSC,sc_isEB,sc_iEtaOrX,sc_iPhiOrY,sc_iEtaMod5,sc_iPhiMod2,sc_iEtaMod20,sc_iPhiMod20,ssFull_e3x3,ssFull_e5x5,ssFull_eMax,ssFull_e2nd,ssFull_sigmaIEtaIEta,ssFull_sigmaIEtaIPhi,ssFull_sigmaIPhiIPhi,ssFull_e2x5Max,ssFull_e2x5Top,ssFull_e2x5Bottom,ssFull_e2x5Left,ssFull_e2x5Right,ssFull_eTop,ssFull_eBottom,ssFull_eLeft,ssFull_eRight,ssFrac_sigmaIEtaIEta,ssFrac_sigmaIPhiIPhi,ele_et,ele_trkEtaMode,ele_trkPhiMode,ele_trkPMode,ele_trkPModeErr,ele_fbrem,ele_hademTow,ele_ecalDrivenSeed,ele_nrSatCrys,mc_energy
count,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,14985167.0,1.498517e+07
mean,1.997382e+01,5.046470e+06,3.779257e+02,7.335956e+00,1.193437e-02,2.406599e-02,3.728392e+02,2.432788e+00,-2.997636e-06,-5.677242e-06,5.742211e-01,2.145832e+01,1.252997e+02,5.020965e-04,2.873538e-01,-4.670685e-03,5.463315e+00,3.581909e+02,3.744382e+02,2.350957e+02,6.351256e+01,1.546967e-02,-1.061414e-02,2.014904e-02,3.543160e+02,4.613595e+01,4.604879e+01,4.064065e+01,4.082364e+01,2.664194e+01,2.661393e+01,2.227410e+01,2.236086e+01,1.533254e-02,1.929471e-02,1.519284e+02,-1.247946e-02,-1.745397e-02,7.617798e+02,3.168371e+07,4.421127e-01,8.037200e-03,9.691119e-01,0.0,3.968549e+02
std,1.276845e+01,2.889091e+06,3.478749e+02,1.194761e+01,5.672930e-03,1.585948e-02,3.486078e+02,1.799880e+00,1.345036e-03,6.332732e-03,4.944606e-01,4.701327e+01,1.030828e+02,1.846125e+00,4.525280e-01,8.109182e+00,6.444108e+00,3.388517e+02,3.488895e+02,2.470245e+02,7.259937e+01,7.874400e-03,1.717326e-01,1.077849e-02,3.320492e+02,6.396088e+01,6.391234e+01,6.351047e+01,6.354219e+01,4.571831e+01,4.569967e+01,4.454048e+01,4.453814e+01,7.624125e-03,9.973876e-03,8.470121e+01,1.502251e+00,1.803607e+00,2.471915e+05,1.073087e+11,2.442862e+00,4.315716e-01,1.730147e-01,0.0,3.592299e+02
min,0.000000e+00,1.000000e+00,2.808963e-01,0.000000e+00,8.671309e-05,1.403324e-06,2.808963e-01,1.000000e+00,-2.552902e-01,-3.811861e-01,0.000000e+00,-8.500000e+01,1.000000e+00,-4.000000e+00,0.000000e+00,-1.900000e+01,0.000000e+00,4.548348e-01,4.548348e-01,2.317474e-01,1.494172e-01,0.000000e+00,-1.000000e+00,2.690318e-04,4.298494e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.035868e-10,2.194893e-09,2.162664e-02,-4.842429e+00,-3.141592e+00,3.589295e-02,1.568661e-05,-6.814325e+03,0.000000e+00,0.000000e+00,0.0,1.120957e+00
25%,9.258972e+00,2.537200e+06,1.361745e+02,0.000000e+00,7.763034e-03,1.356573e-02,1.312442e+02,1.000000e+00,-7.104153e-05,-3.646433e-04,0.000000e+00,-1.100000e+01,4.400000e+01,0.000000e+00,0.000000e+00,-2.000000e+00,0.000000e+00,1.250329e+02,1.335008e+02,7.157972e+01,1.811228e+01,8.913856e-03,-9.280901e-02,1.086688e-02,1.256941e+02,1.085857e+01,1.079410e+01,7.249640e+00,7.366873e+00,4.211712e+00,4.200323e+00,2.424657e+00,2.471955e+00,8.932089e-03,1.073510e-02,7.868557e+01,-1.294079e+00,-1.580388e+00,1.098301e+02,9.068682e+00,2.002993e-01,0.000000e+00,1.000000e+00,0.0,1.465395e+02
50%,1.942506e+01,5.055925e+06,2.684621e+02,0.000000e+00,9.724634e-03,2.099512e-02,2.639473e+02,2.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,3.200000e+01,7.800000e+01,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,2.511379e+02,2.644055e+02,1.536150e+02,4.005738e+01,9.519991e-03,-5.214928e-03,1.623179e-02,2.500212e+02,2.502279e+01,2.496968e+01,1.839296e+01,1.861379e+01,1.096402e+01,1.095723e+01,7.024039e+00,7.125227e+00

In [126]:
#Some outlier?!
mydf['ssFull_e5x5'].min()

-7.665816

In [ ]:
import matplotlib as plt
df = pd.DataFrame()
df['ssPositive'] = mydf.ssFull_e5x5.where(mydf.ssFull_e5x5 >= 0)
df['ssNegative'] = mydf.ssFull_e5x5.where(mydf.ssFull_e5x5 < 0)
df

In [15]:
df[df['ssNegative'].notnull()]

,ssPositive,ssNegative
356084,NaN,-7.665816


In [11]:
usage()

2971.625

In [127]:
#remove one single outlier
mydf = mydf[mydf.ssFull_e5x5>0]


In [7]:
mydf.describe()

,rho,evt_eventnr,sc_rawEnergy,sc_rawESEnergy,sc_etaWidth,sc_phiWidth,sc_seedClusEnergy,sc_numberOfClusters,sc_dEtaSeedSC,sc_dPhiSeedSC,...,ele_et,ele_trkEtaMode,ele_trkPhiMode,ele_trkPMode,ele_trkPModeErr,ele_fbrem,ele_hademTow,ele_ecalDrivenSeed,ele_nrSatCrys,mc_energy
count,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,...,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,14985167.0,1.498517e+07
mean,1.997382e+01,5.046470e+06,3.779257e+02,7.335956e+00,1.193437e-02,2.406599e-02,3.728392e+02,2.432788e+00,-2.997636e-06,-5.677242e-06,...,1.519284e+02,-1.247946e-02,-1.745397e-02,7.617798e+02,3.168371e+07,4.421127e-01,8.037200e-03,9.691119e-01,0.0,3.968549e+02
std,1.276845e+01,2.889091e+06,3.478749e+02,1.194761e+01,5.672930e-03,1.585948e-02,3.486078e+02,1.799880e+00,1.345036e-03,6.332732e-03,...,8.470121e+01,1.502251e+00,1.803607e+00,2.471915e+05,1.073087e+11,2.442862e+00,4.315716e-01,1.730147e-01,0.0,3.592299e+02
min,0.000000e+00,1.000000e+00,2.808963e-01,0.000000e+00,8.671309e-05,1.403324e-06,2.808963e-01,1.000000e+00,-2.552902e-01,-3.811861e-01,...,2.162664e-02,-4.842429e+00,-3.141592e+00,3.589295e-02,1.568661e-05,-6.814325e+03,0.000000e+00,0.000000e+00,0.0,1.120957e+00
25%,9.258972e+00,2.537200e+06,1.361745e+02,0.000000e+00,7.763034e-03,1.356573e-02,1.312442e+02,1.000000e+00,-7.104153e-05,-3.646433e-04,...,7.868557e+01,-1.294079e+00,-1.580388e+00,1.098301e+02,9.068682e+00,2.002993e-01,0.000000e+00,1.000000e+00,0.0,1.465395e+02
50%,1.942506e+01,5.055925e+06,2.684621e+02,0.000000e+00,9.724634e-03,2.099512e-02,2.639473e+02,2.000000e+00,0.000000e+00,0.000000e+00,...,1.515566e+02,-1.502267e-02,-2.160997e-02,2.306579e+02,2.805947e+01,4.727513e-01,0.000000e+00,1.000000e+00,0.0,2.802458e+02
75%,2.959096e+01,7.559198e+06,4.985067e+02,1.303652e+01,1.511821e-02,2.878841e-02,4.899037e+02,3.000000e+00,6.925964e-05,3.558224e-04,...,2.250044e+02,1.268709e+00,1.534383e+00,4.311000e+02,1.013056e+02,7.424718e-01,2.412337e-03,1.000000e+00,0.0,5.278936e+02
max,9.160725e+01,1.000000e+07,2.480208e+03,1.210290e+02,3.414597e-01,8.972858e-01,2.480208e+03,5.400000e+01,2.174096e-01,3.975364e-01,...,2.055027e+03,5.286520e+00,3.141592e+00,9.124867e+08,4.149836e+14,9.998899e-01,2.404923e+02,1.000000e+00,0.0,2.695155e+03


### Define lists of barrel and endcap variables for training, also add to them 4 additional important variables.

In [157]:
barrel_vars = [
'mc_energy',
'sc_rawESEnergy',
    
'meas_energy',
'Emc_over_Emeas',

    
'sc_rawEnergy',
'sc_etaWidth',
'sc_phiWidth',
'sc_seedClusEnergy_over_sc_rawEnergy',
'ssFull_e5x5_over_sc_rawEnergy',
'ele_hademTow',
'rho',
'sc_dEtaSeedSC',
'sc_dPhiSeedSC',
'ssFull_e3x3_over_sc_rawEnergy',
'ssFull_sigmaIEtaIEta',
'ssFull_sigmaIEtaIPhi',
'ssFull_sigmaIPhiIPhi',
'ssFull_eMax_over_ssFull_e5x5',
'ssFull_e2nd_over_ssFull_e5x5',
'ssFull_eTop_over_ssFull_e5x5',
'ssFull_eBottom_over_ssFull_e5x5',
'ssFull_eLeft_over_ssFull_e5x5',
'ssFull_eRight_over_ssFull_e5x5',
'ssFull_e2x5Max_over_ssFull_e5x5',
'ssFull_e2x5Left_over_ssFull_e5x5',
'ssFull_e2x5Right_over_ssFull_e5x5',
'ssFull_e2x5Top_over_ssFull_e5x5',
'ssFull_e2x5Bottom_over_ssFull_e5x5',
'ele_nrSatCrys',
'sc_numberOfClusters',
'sc_iEtaOrX',
'sc_iPhiOrY',
'sc_iEtaMod5',
'sc_iPhiMod2',
'sc_iEtaMod20',
'sc_iPhiMod20'
]
len(barrel_vars)

36

In [158]:
endcap_vars = [
'mc_energy',
'sc_rawESEnergy',

'meas_energy',
'Emc_over_Emeas',
    
'sc_rawEnergy',
'sc_etaWidth',
'sc_phiWidth',
'sc_seedClusEnergy_over_sc_rawEnergy',
'ssFull_e5x5_over_sc_rawEnergy',
'ele_hademTow',
'rho',
'sc_dEtaSeedSC',
'sc_dPhiSeedSC',
'ssFull_e3x3_over_sc_rawEnergy',
'ssFull_sigmaIEtaIEta',
'ssFull_sigmaIEtaIPhi',
'ssFull_sigmaIPhiIPhi',
'ssFull_eMax_over_ssFull_e5x5',
'ssFull_e2nd_over_ssFull_e5x5',
'ssFull_eTop_over_ssFull_e5x5',
'ssFull_eBottom_over_ssFull_e5x5',
'ssFull_eLeft_over_ssFull_e5x5',
'ssFull_eRight_over_ssFull_e5x5',
'ssFull_e2x5Max_over_ssFull_e5x5',
'ssFull_e2x5Left_over_ssFull_e5x5',
'ssFull_e2x5Right_over_ssFull_e5x5',
'ssFull_e2x5Top_over_ssFull_e5x5',
'ssFull_e2x5Bottom_over_ssFull_e5x5',
'ele_nrSatCrys',
'sc_numberOfClusters',
'sc_iEtaOrX',
'sc_iPhiOrY',
'sc_rawESEnergy_over_sc_rawEnergy'
    
]
len(endcap_vars)

33

### Create exact version of variables to be used in the training.

In [130]:
mydf['sc_seedClusEnergy_over_sc_rawEnergy'] = mydf['sc_seedClusEnergy']/mydf['sc_rawEnergy']
mydf['ssFull_e5x5_over_sc_rawEnergy'] = mydf['ssFull_e5x5']/mydf['sc_rawEnergy']

mydf['ssFull_e3x3_over_sc_rawEnergy'] = mydf['ssFull_e3x3']/mydf['sc_rawEnergy']

mydf['ssFull_eMax_over_ssFull_e5x5'] = mydf['ssFull_eMax']/mydf['ssFull_e5x5']
mydf['ssFull_e2nd_over_ssFull_e5x5'] = mydf['ssFull_e2nd']/mydf['ssFull_e5x5']
mydf['ssFull_eTop_over_ssFull_e5x5'] = mydf['ssFull_eTop']/mydf['ssFull_e5x5']
mydf['ssFull_eBottom_over_ssFull_e5x5'] = mydf['ssFull_eBottom']/mydf['ssFull_e5x5']
mydf['ssFull_eLeft_over_ssFull_e5x5'] = mydf['ssFull_eLeft']/mydf['ssFull_e5x5']
mydf['ssFull_eRight_over_ssFull_e5x5'] = mydf['ssFull_eRight']/mydf['ssFull_e5x5']

mydf['ssFull_e2x5Max_over_ssFull_e5x5'] = mydf['ssFull_e2x5Max']/mydf['ssFull_e5x5']
mydf['ssFull_e2x5Left_over_ssFull_e5x5'] = mydf['ssFull_e2x5Left']/mydf['ssFull_e5x5']
mydf['ssFull_e2x5Right_over_ssFull_e5x5'] = mydf['ssFull_e2x5Right']/mydf['ssFull_e5x5']
mydf['ssFull_e2x5Top_over_ssFull_e5x5'] = mydf['ssFull_e2x5Top']/mydf['ssFull_e5x5']
mydf['ssFull_e2x5Bottom_over_ssFull_e5x5'] = mydf['ssFull_e2x5Bottom']/mydf['ssFull_e5x5']

mydf['sc_rawESEnergy_over_sc_rawEnergy'] = mydf['sc_rawESEnergy']/mydf['sc_rawEnergy']

/usr/local/bin/start_ipykernel.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/bin/start_ipykernel.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/bin/start_ipykernel.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/bin/start_ipyke

In [131]:
usage()

6697.32421875

In [16]:
mydf.describe()

,rho,evt_eventnr,sc_rawEnergy,sc_rawESEnergy,sc_etaWidth,sc_phiWidth,sc_seedClusEnergy,sc_numberOfClusters,sc_dEtaSeedSC,sc_dPhiSeedSC,sc_isEB,sc_iEtaOrX,sc_iPhiOrY,sc_iEtaMod5,sc_iPhiMod2,sc_iEtaMod20,sc_iPhiMod20,ssFull_e3x3,ssFull_e5x5,ssFull_eMax,ssFull_e2nd,ssFull_sigmaIEtaIEta,ssFull_sigmaIEtaIPhi,ssFull_sigmaIPhiIPhi,ssFull_e2x5Max,ssFull_e2x5Top,ssFull_e2x5Bottom,ssFull_e2x5Left,ssFull_e2x5Right,ssFull_eTop,ssFull_eBottom,ssFull_eLeft,ssFull_eRight,ssFrac_sigmaIEtaIEta,ssFrac_sigmaIPhiIPhi,ele_et,ele_trkEtaMode,ele_trkPhiMode,ele_trkPMode,ele_trkPModeErr,ele_fbrem,ele_hademTow,ele_ecalDrivenSeed,ele_nrSatCrys,mc_energy,sc_seedClusEnergy_over_sc_rawEnergy,ssFull_e5x5_over_sc_rawEnergy,ssFull_e3x3_over_sc_rawEnergy,ssFull_eMax_over_ssFull_e5x5,ssFull_e2nd_over_ssFull_e5x5,ssFull_eTop_over_ssFull_e5x5,ssFull_eBottom_over_ssFull_e5x5,ssFull_eLeft_over_ssFull_e5x5,ssFull_eRight_over_ssFull_e5x5,ssFull_e2x5Max_over_ssFull_e5x5,ssFull_e2x5Left_over_ssFull_e5x5,ssFull_e2x5Right_over_ssFull_e5x5,ssFull_e2x5Top_over_ssFull_e5x5,ssFull_e2x5Bottom_over_ssFull_e5x5,sc_rawESEnergy_over_sc_rawEnergy
count,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,14985167.0,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07,1.498517e+07
mean,1.997382e+01,5.046470e+06,3.779257e+02,7.335956e+00,1.193437e-02,2.406599e-02,3.728392e+02,2.432788e+00,-2.997636e-06,-5.677242e-06,5.742211e-01,2.145832e+01,1.252997e+02,5.020965e-04,2.873538e-01,-4.670685e-03,5.463315e+00,3.581909e+02,3.744382e+02,2.350957e+02,6.351256e+01,1.546967e-02,-1.061414e-02,2.014904e-02,3.543160e+02,4.613595e+01,4.604879e+01,4.064065e+01,4.082364e+01,2.664194e+01,2.661393e+01,2.227410e+01,2.236086e+01,1.533254e-02,1.929471e-02,1.519284e+02,-1.247946e-02,-1.745397e-02,7.617798e+02,3.168371e+07,4.421127e-01,8.037200e-03,9.691119e-01,0.0,3.968549e+02,9.650916e-01,9.831990e-01,9.267472e-01,5.929695e-01,1.795003e-01,8.042685e-02,8.038022e-02,5.788452e-02,5.825862e-02,9.430997e-01,1.122208e-01,1.130110e-01,1.419773e-01,1.416803e-01,1.761959e-02
std,1.276845e+01,2.889091e+06,3.478749e+02,1.194761e+01,5.672930e-03,1.585948e-02,3.486078e+02,1.799880e+00,1.345036e-03,6.332732e-03,4.944606e-01,4.701327e+01,1.030828e+02,1.846125e+00,4.525280e-01,8.109182e+00,6.444108e+00,3.388517e+02,3.488895e+02,2.470245e+02,7.259937e+01,7.874400e-03,1.717326e-01,1.077849e-02,3.320492e+02,6.396088e+01,6.391234e+01,6.351047e+01,6.354219e+01,4.571831e+01,4.569967e+01,4.454048e+01,4.453814e+01,7.624125e-03,9.973876e-03,8.470121e+01,1.502251e+00,1.803607e+00,2.471915e+05,1.073087e+11,2.442862e+00,4.315716e-01,1.730147e-01,0.0,3.592299e+02,8.551732e-02,4.866453e-01,3.177953e-01,1.430795e-01,9.528822e-02,8.948058e-02,8.948939e-02,7.630977e-02,7.633292e-02,3.474898e-02,1.124365e-01,1.125588e-01,1.239299e-01,1.238616e-01,3.628573e-02
min,0.000000e+00,1.000000e+00,2.808963e-01,0.000000e+00,8.671309e-05,1.403324e-06,2.808963e-01,1.000000e+00,-2.552902e-01,-3.811861e-01,0.000000e+00,-8.500000e+01,1.000000e+00,-4.000000e+00,0.000000e+00,-1.900000e+01,0.000000e+00,4.548348e-01,4.548348e-01,2.317474e-01,1.494172e-01,0.000000e+00,-1.000000e+00,2.690318e-04,4.298494e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.035868e-10,2.194893e-09,2.162664e-02,-4.842429e+00,-3.141592e+00,3.589295e-02,1.568661e-05,-6.814325e+03,0.0

In [132]:
mydf.isnull().values.any()

False

### Define measured energy as a sum of ECAL and Preshower energies, also define a target ratio of energies as MC true energy over the measured energy

In [133]:
mydf['meas_energy'] = (mydf['sc_rawEnergy']+mydf['sc_rawESEnergy'])

/usr/local/bin/start_ipykernel.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [134]:
mydf['meas_energy'].min() 

0.2808963

In [135]:
mydf['Emc_over_Emeas'] = mydf['mc_energy']/mydf['meas_energy']
mydf['Emc_over_Emeas'].min()

/usr/local/bin/start_ipykernel.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0.022844527

In [136]:
mydf['mc_energy'].min() 


1.120957

In [137]:
mydf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14985167 entries, 0 to 14985167
Data columns (total 62 columns):
rho                                    float32
evt_eventnr                            int32
sc_rawEnergy                           float32
sc_rawESEnergy                         float32
sc_etaWidth                            float32
sc_phiWidth                            float32
sc_seedClusEnergy                      float32
sc_numberOfClusters                    float32
sc_dEtaSeedSC                          float32
sc_dPhiSeedSC                          float32
sc_isEB                                float32
sc_iEtaOrX                             float32
sc_iPhiOrY                             float32
sc_iEtaMod5                            float32
sc_iPhiMod2                            float32
sc_iEtaMod20                           float32
sc_iPhiMod20                           float32
ssFull_e3x3                            float32
ssFull_e5x5                            fl

In [159]:
usage()

13566.8359375

In [160]:
len(barrel_vars)

36

In [161]:
barrel_vars

['mc_energy',
 'sc_rawESEnergy',
 'meas_energy',
 'Emc_over_Emeas',
 'sc_rawEnergy',
 'sc_etaWidth',
 'sc_phiWidth',
 'sc_seedClusEnergy_over_sc_rawEnergy',
 'ssFull_e5x5_over_sc_rawEnergy',
 'ele_hademTow',
 'rho',
 'sc_dEtaSeedSC',
 'sc_dPhiSeedSC',
 'ssFull_e3x3_over_sc_rawEnergy',
 'ssFull_sigmaIEtaIEta',
 'ssFull_sigmaIEtaIPhi',
 'ssFull_sigmaIPhiIPhi',
 'ssFull_eMax_over_ssFull_e5x5',
 'ssFull_e2nd_over_ssFull_e5x5',
 'ssFull_eTop_over_ssFull_e5x5',
 'ssFull_eBottom_over_ssFull_e5x5',
 'ssFull_eLeft_over_ssFull_e5x5',
 'ssFull_eRight_over_ssFull_e5x5',
 'ssFull_e2x5Max_over_ssFull_e5x5',
 'ssFull_e2x5Left_over_ssFull_e5x5',
 'ssFull_e2x5Right_over_ssFull_e5x5',
 'ssFull_e2x5Top_over_ssFull_e5x5',
 'ssFull_e2x5Bottom_over_ssFull_e5x5',
 'ele_nrSatCrys',
 'sc_numberOfClusters',
 'sc_iEtaOrX',
 'sc_iPhiOrY',
 'sc_iEtaMod5',
 'sc_iPhiMod2',
 'sc_iEtaMod20',
 'sc_iPhiMod20']

In [162]:
len(endcap_vars)

33

### Subset the dataframe and keep only barrel information.

In [143]:
barrel_df = mydf[(mydf.sc_isEB==1)]

In [144]:
barrel_df.sc_isEB.min()

1.0

In [146]:
barrel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8604800 entries, 0 to 14985167
Data columns (total 62 columns):
rho                                    float32
evt_eventnr                            int32
sc_rawEnergy                           float32
sc_rawESEnergy                         float32
sc_etaWidth                            float32
sc_phiWidth                            float32
sc_seedClusEnergy                      float32
sc_numberOfClusters                    float32
sc_dEtaSeedSC                          float32
sc_dPhiSeedSC                          float32
sc_isEB                                float32
sc_iEtaOrX                             float32
sc_iPhiOrY                             float32
sc_iEtaMod5                            float32
sc_iPhiMod2                            float32
sc_iEtaMod20                           float32
sc_iPhiMod20                           float32
ssFull_e3x3                            float32
ssFull_e5x5                            flo

### Store a 'big' barrel dataframe. Contains additional variables which may be needed for cross checking results. Already after preselection and is easy to load in.

In [148]:
barrel_df.columns = barrel_df.columns.astype(str)
print 'Write this big barrel dataframe to disk'

print 'Processing feather'
feather.write_feather(barrel_df, 'big_barrel_df.feather')

print 'Processing parquet'
barrel_df.to_parquet('big_barrel_df.parquet', engine='fastparquet', compression='gzip')

print 'All done!'

Write this big barrel dataframe to disk
Processing feather
Processing parquet
All done!


In [48]:
list(set(mydf.columns).symmetric_difference(set(barrel_vars)))

[u'ssFrac_sigmaIEtaIEta',
 u'ele_trkPhiMode',
 u'ssFull_eBottom',
 'iEtaMod20_sc',
 'meas_energy',
 u'ele_fbrem',
 u'mc_energy',
 u'ele_trkPModeErr',
 u'ssFull_eMax',
 u'ele_trkEtaMode',
 u'ssFull_e2x5Right',
 'iEtaMod5_sc',
 u'ele_trkPMode',
 'iPhiOrY_sc',
 u'ssFull_e2x5Max',
 u'ssFull_eLeft',
 u'ssFull_e2x5Top',
 'sc_rawESEnergy_over_sc_rawEnergy',
 u'sc_iPhiOrY',
 u'ssFull_e2x5Bottom',
 u'ssFrac_sigmaIPhiIPhi',
 u'ssFull_e2x5Left',
 u'ssFull_e5x5',
 u'ssFull_e2nd',
 u'evt_eventnr',
 u'sc_iEtaMod5',
 'iPhiMod2_sc',
 u'ele_ecalDrivenSeed',
 u'ssFull_eRight',
 u'ssFull_e3x3',
 u'ssFull_eTop',
 u'sc_rawESEnergy',
 u'sc_iPhiMod2',
 u'ele_et',
 u'sc_iEtaMod20',
 u'sc_isEB',
 u'sc_seedClusEnergy']

In [52]:
list(set(barrel_vars).difference(set(mydf.columns)))

['iEtaMod20_sc', 'iEtaMod5_sc', 'iPhiMod2_sc', 'iPhiOrY_sc']

In [149]:
len(list(set(mydf.columns).intersection(set(barrel_vars))))

35

### Define a barrel dataframe with variables important for training (using a dedicated notebook), and store this dataframe.

In [163]:
final_barrel_df = barrel_df[barrel_df.columns & barrel_vars]

In [164]:
usage()

13600.6875

In [165]:
final_barrel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8604800 entries, 0 to 14985167
Data columns (total 36 columns):
rho                                    float32
sc_rawEnergy                           float32
sc_rawESEnergy                         float32
sc_etaWidth                            float32
sc_phiWidth                            float32
sc_numberOfClusters                    float32
sc_dEtaSeedSC                          float32
sc_dPhiSeedSC                          float32
sc_iEtaOrX                             float32
sc_iPhiOrY                             float32
sc_iEtaMod5                            float32
sc_iPhiMod2                            float32
sc_iEtaMod20                           float32
sc_iPhiMod20                           float32
ssFull_sigmaIEtaIEta                   float32
ssFull_sigmaIEtaIPhi                   float32
ssFull_sigmaIPhiIPhi                   float32
ele_hademTow                           float32
ele_nrSatCrys                          f

In [166]:
final_barrel_df.describe()

,rho,sc_rawEnergy,sc_rawESEnergy,sc_etaWidth,sc_phiWidth,sc_numberOfClusters,sc_dEtaSeedSC,sc_dPhiSeedSC,sc_iEtaOrX,sc_iPhiOrY,sc_iEtaMod5,sc_iPhiMod2,sc_iEtaMod20,sc_iPhiMod20,ssFull_sigmaIEtaIEta,ssFull_sigmaIEtaIPhi,ssFull_sigmaIPhiIPhi,ele_hademTow,ele_nrSatCrys,mc_energy,sc_seedClusEnergy_over_sc_rawEnergy,ssFull_e5x5_over_sc_rawEnergy,ssFull_e3x3_over_sc_rawEnergy,ssFull_eMax_over_ssFull_e5x5,ssFull_e2nd_over_ssFull_e5x5,ssFull_eTop_over_ssFull_e5x5,ssFull_eBottom_over_ssFull_e5x5,ssFull_eLeft_over_ssFull_e5x5,ssFull_eRight_over_ssFull_e5x5,ssFull_e2x5Max_over_ssFull_e5x5,ssFull_e2x5Left_over_ssFull_e5x5,ssFull_e2x5Right_over_ssFull_e5x5,ssFull_e2x5Top_over_ssFull_e5x5,ssFull_e2x5Bottom_over_ssFull_e5x5,meas_energy,Emc_over_Emeas
count,8.604800e+06,8.604800e+06,8604800.0,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8604800.0,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06,8.604800e+06
mean,1.993312e+01,2.088408e+02,0.0,8.282442e-03,2.267043e-02,2.819328e+00,-6.000560e-07,-8.303765e-06,-1.041085e-01,1.807342e+02,8.743957e-04,5.004235e-01,-8.133949e-03,9.514298e+00,8.967754e-03,-3.373731e-03,1.245964e-02,3.639736e-03,0.0,2.156655e+02,9.556687e-01,9.719350e-01,9.127039e-01,5.814700e-01,1.848440e-01,9.064155e-02,9.060916e-02,5.227534e-02,5.290156e-02,9.520532e-01,1.033461e-01,1.046839e-01,1.541710e-01,1.537542e-01,2.088408e+02,1.078456e+00
std,1.276103e+01,1.336567e+02,0.0,1.810990e-03,1.767361e-02,2.053480e+00,1.302780e-03,6.644111e-03,4.894718e+01,1.045375e+02,2.436248e+00,4.999999e-01,1.070132e+01,5.811683e+00,9.460002e-04,9.930093e-02,2.934863e-03,3.363293e-01,0.0,1.362011e+02,9.621000e-02,1.575218e-01,1.439846e-01,1.402599e-01,9.465282e-02,9.330297e-02,9.329543e-02,7.224157e-02,7.229788e-02,1.397212e-02,1.081271e-01,1.084513e-01,1.269594e-01,1.269048e-01,1.336567e+02,1.258465e+00
min,0.000000e+00,2.808963e-01,0.0,2.126136e-04,4.063789e-05,1.000000e+00,-9.987355e-02,-3.811861e-01,-8.500000e+01,1.000000e+00,-4.000000e+00,0.000000e+00,-1.900000e+01,0.000000e+00,0.000000e+00,-1.000000e+00,2.690318e-04,0.000000e+00,0.0,1.120957e+00,1.644615e-01,1.988927e-01,1.545707e-01,5.239881e-03,4.797683e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.298138e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.808963e-01,1.287299e-01
25%,9.218432e+00,1.002411e+02,0.0,7.217530e-03,1.040706e-02,1.000000e+00,-1.002429e-04,-5.385262e-04,-4.200000e+01,9.000000e+01,-2.000000e+00,0.000000e+00,-9.000000e+00,4.000000e+00,8.719118e-03,-5.687158e-02,1.033538e-02,0.000000e+00,0.0,1.059380e+02,9.619800e-01,9.679362e-01,9.021573e-01,4.708541e-01,1.048821e-01,2.210198e-02,2.204533e-02,1.086841e-02,1.112711e-02,9.455189e-01,3.401419e-02,3.469518e-02,5.524228e-02,5.486086e-02,1.002411e+02,1.005088e+00
50%,1.937348e+01,1.954058e+02,0.0,8.037896e-03,1.705686e-02,2.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,1.810000e+02,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+01,8.992155e-03,-1.011613e-03,1.122502e-02,0.000000e+00,0.0,2.020149e+02,9.940259e-01,9.915215e-01,9.468320e-01,5.934177e-01,1.721211e-01,5.159103e-02,5.159243e-02,2.175028e-02,2.237645e-02,9.539108e-01,5.876791e-02,6.014754e-02,1.062083e-01,1.058485e-01,1.954058e+02,1.017478e+00
75%,2.954667e+01,2.926260e+02,0.0,9.288809e-03,2.868558e-02,4.000000e+00,9.807901e-05,5.250425e-04,4.200000e+01,2.720000e+02,2.000000e+00,1.000000e+00,9.000000e+00,1.500000e+01,9.309629e-03,5.199568e-02,1.413548e-02,0.000000e+00,0.0,2.984509e+02,1.000000e+00,9.987040e-01,9.608448e-01,7.020959e-01,2.540470e-01,1.280296e-01,1.280553e-01,5.652317e-02,5.794043e-02,9.593377e-01,1.234920e-01,1.259224e-01,2.189473e-01,2.184005e-01,2.926260e+02,1.044554e+00
max,8.482404e+01,6.862073e+02,0.0,1.966752

In [167]:
usage()

13601.58203125

In [168]:
final_barrel_df.isnull().values.any()

False

In [76]:
final_barrel_df.columns = final_barrel_df.columns.astype(str)

### Store the final barrel dataframe.

In [169]:
%time
print 'Write this final barrel dataframe to disk'

print 'Processing feather'
feather.write_feather(final_barrel_df, 'final_barrel_df.feather')

print 'Processing parquet'
final_barrel_df.to_parquet('final_barrel_df.parquet', engine='fastparquet', compression='gzip')

print 'All done!'

CPU times: user 18 µs, sys: 5 µs, total: 23 µs
Wall time: 41 µs
Write this final barrel dataframe to disk
Processing feather
Processing parquet
All done!


In [170]:
usage()

13610.703125

### Repeat the same logic with endcap: subset endcap information and store it.

In [171]:
endcap_df = mydf[(mydf.sc_isEB==0)]

In [172]:
endcap_df.sc_isEB.max()

0.0

In [178]:
endcap_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6380367 entries, 7 to 14985165
Data columns (total 62 columns):
rho                                    float32
evt_eventnr                            int32
sc_rawEnergy                           float32
sc_rawESEnergy                         float32
sc_etaWidth                            float32
sc_phiWidth                            float32
sc_seedClusEnergy                      float32
sc_numberOfClusters                    float32
sc_dEtaSeedSC                          float32
sc_dPhiSeedSC                          float32
sc_isEB                                float32
sc_iEtaOrX                             float32
sc_iPhiOrY                             float32
sc_iEtaMod5                            float32
sc_iPhiMod2                            float32
sc_iEtaMod20                           float32
sc_iPhiMod20                           float32
ssFull_e3x3                            float32
ssFull_e5x5                            flo

### We do not need anymore barrel dataframes in this notebook, can safely remove them from the RAM.

In [174]:
del barrel_df

In [175]:
usage()

13094.02734375

In [176]:
del final_barrel_df

In [177]:
usage()

11912.3359375

In [179]:
endcap_df.columns = endcap_df.columns.astype(str)
print 'Write this big endcap dataframe to disk'

print 'Processing feather'
feather.write_feather(endcap_df, 'big_endcap_df.feather')

print 'Processing parquet'
endcap_df.to_parquet('big_endcap_df.parquet', engine='fastparquet', compression='gzip')

print 'All done!'

Write this big endcap dataframe to disk
Processing feather
Processing parquet
All done!


In [180]:
usage()

11918.03515625

In [181]:
final_endcap_df = endcap_df[endcap_df.columns & endcap_vars]

In [182]:
final_endcap_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6380367 entries, 7 to 14985165
Data columns (total 33 columns):
rho                                    float32
sc_rawEnergy                           float32
sc_rawESEnergy                         float32
sc_etaWidth                            float32
sc_phiWidth                            float32
sc_numberOfClusters                    float32
sc_dEtaSeedSC                          float32
sc_dPhiSeedSC                          float32
sc_iEtaOrX                             float32
sc_iPhiOrY                             float32
ssFull_sigmaIEtaIEta                   float32
ssFull_sigmaIEtaIPhi                   float32
ssFull_sigmaIPhiIPhi                   float32
ele_hademTow                           float32
ele_nrSatCrys                          float32
mc_energy                              float32
sc_seedClusEnergy_over_sc_rawEnergy    float32
ssFull_e5x5_over_sc_rawEnergy          float32
ssFull_e3x3_over_sc_rawEnergy          f

In [183]:
final_endcap_df.describe()

,rho,sc_rawEnergy,sc_rawESEnergy,sc_etaWidth,sc_phiWidth,sc_numberOfClusters,sc_dEtaSeedSC,sc_dPhiSeedSC,sc_iEtaOrX,sc_iPhiOrY,ssFull_sigmaIEtaIEta,ssFull_sigmaIEtaIPhi,ssFull_sigmaIPhiIPhi,ele_hademTow,ele_nrSatCrys,mc_energy,sc_seedClusEnergy_over_sc_rawEnergy,ssFull_e5x5_over_sc_rawEnergy,ssFull_e3x3_over_sc_rawEnergy,ssFull_eMax_over_ssFull_e5x5,ssFull_e2nd_over_ssFull_e5x5,ssFull_eTop_over_ssFull_e5x5,ssFull_eBottom_over_ssFull_e5x5,ssFull_eLeft_over_ssFull_e5x5,ssFull_eRight_over_ssFull_e5x5,ssFull_e2x5Max_over_ssFull_e5x5,ssFull_e2x5Left_over_ssFull_e5x5,ssFull_e2x5Right_over_ssFull_e5x5,ssFull_e2x5Top_over_ssFull_e5x5,ssFull_e2x5Bottom_over_ssFull_e5x5,sc_rawESEnergy_over_sc_rawEnergy,meas_energy,Emc_over_Emeas
count,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6380367.0,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06,6.380367e+06
mean,2.002875e+01,6.059588e+02,1.722951e+01,1.685951e-02,2.594809e-02,1.911485e+00,-6.231104e-06,-2.135019e-06,5.053819e+01,5.053917e+01,2.423840e-02,-2.037881e-02,3.051924e-02,1.396778e-02,0.0,6.412155e+02,9.777978e-01,9.983891e-01,9.456851e-01,6.084775e-01,1.722936e-01,6.665095e-02,6.658509e-02,6.544929e-02,6.548342e-02,9.310262e-01,1.241894e-01,1.242414e-01,1.255324e-01,1.253971e-01,4.138204e-02,6.231884e+02,1.151106e+00
std,1.277826e+01,4.117969e+02,1.283730e+01,5.377533e-03,1.277936e-02,1.203495e+00,1.399998e-03,5.886765e-03,2.207478e+01,2.202891e+01,3.243336e-03,2.362239e-01,8.599845e-03,5.336924e-01,0.0,4.172255e+02,6.641851e-02,7.227359e-01,4.567427e-01,1.453626e-01,9.566692e-02,8.206116e-02,8.208965e-02,8.086134e-02,8.089892e-02,4.815415e-02,1.169375e-01,1.169364e-01,1.177397e-01,1.176892e-01,4.592390e-02,4.176558e+02,2.477922e+00
min,0.000000e+00,7.550039e-01,0.000000e+00,8.671309e-05,1.403324e-06,1.000000e+00,-2.552902e-01,-2.876483e-01,1.000000e+00,1.000000e+00,5.438504e-04,-9.974241e-01,3.754915e-04,0.000000e+00,0.0,2.422040e+00,1.797029e-01,9.955423e-02,9.955423e-02,1.882909e-03,1.653227e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.797474e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.550039e-01,2.284453e-02
25%,9.314892e+00,2.728871e+02,7.855617e+00,1.293468e-02,1.893865e-02,1.000000e+00,-1.793936e-05,-8.068089e-05,3.300000e+01,3.300000e+01,2.278456e-02,-1.887045e-01,2.474766e-02,0.000000e+00,0.0,3.062990e+02,9.870287e-01,9.889480e-01,9.382826e-01,4.955660e-01,9.141396e-02,1.420182e-02,1.417950e-02,1.396071e-02,1.397697e-02,9.256333e-01,4.115809e-02,4.119439e-02,4.167926e-02,4.164112e-02,1.583423e-02,2.852552e+02,1.002864e+00
50%,1.949312e+01,5.480710e+02,1.566993e+01,1.617286e-02,2.321593e-02,2.000000e+00,0.000000e+00,0.000000e+00,5.100000e+01,5.100000e+01,2.392450e-02,-2.344828e-02,2.726633e-02,1.603853e-03,0.0,5.862535e+02,9.990657e-01,9.985538e-01,9.620269e-01,6.293145e-01,1.551048e-01,3.093261e-02,3.086119e-02,3.039867e-02,3.043937e-02,9.445233e-01,7.727589e-02,7.734247e-02,7.814413e-02,7.799799e-02,3.060205e-02,5.654055e+02,1.018023e+00
75%,2.965118e+01,8.576479e+02,2.473589e+01,2.020009e-02,2.885893e-02,2.000000e+00,1.611407e-05,7.099856e-05,6.800000e+01,6.800000e+01,2.505134e-02,1.425038e-01,3.377106e-02,4.809177e-03,0.0,8.969975e+02,1.000000e+00,1.006260e+00,9.741447e-01,7.318827e-01,2.405415e-01,8.333041e-02,8.312925e-02,8.167752e-02,8.156478e-02,9.559149e-01,1.657324e-01,1.658197e-01,1.676858e-01,1.674890e-01,5.297057e-02,8.829196e+02,1.048977e+00
max,9.160725e+01,2.480208e+03,1.210290e+02,3.414597e-01,8.972858e-01,5.400000e+01,2.174096e-01,3.975364e-01,1.000000e+02,1.000000e+02,8.180051e-02,9.994417e-01,9.596091e-02,2.340993e+02,0.0,2.695155e+03,1.000000e+00,6.006783e+02,2.782103e+02,9.718506e-01,8.007877e-01,7.982320e-01,8.2

In [184]:
final_endcap_df.isnull().values.any()

False

In [185]:
final_endcap_df.columns = final_endcap_df.columns.astype(str)

In [187]:
final_endcap_df

,rho,sc_rawEnergy,sc_rawESEnergy,sc_etaWidth,sc_phiWidth,sc_numberOfClusters,sc_dEtaSeedSC,sc_dPhiSeedSC,sc_iEtaOrX,sc_iPhiOrY,ssFull_sigmaIEtaIEta,ssFull_sigmaIEtaIPhi,ssFull_sigmaIPhiIPhi,ele_hademTow,ele_nrSatCrys,mc_energy,sc_seedClusEnergy_over_sc_rawEnergy,ssFull_e5x5_over_sc_rawEnergy,ssFull_e3x3_over_sc_rawEnergy,ssFull_eMax_over_ssFull_e5x5,ssFull_e2nd_over_ssFull_e5x5,ssFull_eTop_over_ssFull_e5x5,ssFull_eBottom_over_ssFull_e5x5,ssFull_eLeft_over_ssFull_e5x5,ssFull_eRight_over_ssFull_e5x5,ssFull_e2x5Max_over_ssFull_e5x5,ssFull_e2x5Left_over_ssFull_e5x5,ssFull_e2x5Right_over_ssFull_e5x5,ssFull_e2x5Top_over_ssFull_e5x5,ssFull_e2x5Bottom_over_ssFull_e5x5,sc_rawESEnergy_over_sc_rawEnergy,meas_energy,Emc_over_Emeas
7,17.500393,107.924393,8.423841,0.012864,0.047712,3.0,-0.002168,6.474053e-03,20.0,27.0,0.021894,-0.061254,0.055772,0.000000,0.0,128.248749,0.763299,0.956247,0.762141,0.496913,0.134050,0.042185,0.017268,0.134050,0.019501,0.820936,0.358051,0.075799,0.265479,0.077578,0.078053,116.348236,1.102284
8,17.500393,108.024261,11.480968,0.014954,0.044356,3.0,0.001522,-3.945758e-03,82.0,74.0,0.022739,0.035947,0.037725,0.000000,0.0,128.248749,0.952532,0.945846,0.884405,0.435059,0.313085,0.044734,0.028915,0.013531,0.313085,0.926243,0.039830,0.442990,0.086960,0.145435,0.106281,119.505226,1.073164
15,20.662251,414.189697,23.596441,0.014047,0.027942,5.0,-0.000815,-1.600069e-03,79.0,72.0,0.023977,0.038398,0.029885,0.000000,0.0,435.324951,0.984393,0.988749,0.947156,0.745048,0.052869,0.026008,0.052869,0.027238,0.052260,0.916417,0.073706,0.094511,0.061845,0.103743,0.056970,437.786133,0.994378
16,20.662251,416.708496,10.159088,0.014249,0.017439,2.0,0.000133,4.568984e-04,22.0,30.0,0.025463,0.289535,0.022554,0.000000,0.0,435.324951,0.998064,1.000397,0.969539,0.566011,0.236400,0.009632,0.236400,0.063468,0.010395,0.957872,0.149547,0.031182,0.021386,0.333354,0.024379,426.867584,1.019813
17,21.473417,55.558941,5.046361,0.010976,0.048473,4.0,-0.002194,1.183567e-03,23.0,83.0,0.028099,-0.296239,0.045582,0.000000,0.0,72.982941,0.929340,0.940300,0.858293,0.757637,0.058510,0.022232,0.058510,0.019532,0.000000,0.924664,0.088491,0.056205,0.064812,0.158020,0.090829,60.605301,1.204234
18,21.473417,56.471313,3.622597,0.008810,0.012283,1.0,0.000000,0.000000e+00,75.0,17.0,0.022231,0.038194,0.029640,0.000000,0.0,72.982941,1.000000,1.037938,0.988445,0.697057,0.088912,0.085239,0.015435,0.014109,0.088912,0.939411,0.045717,0.145400,0.139298,0.049250,0.064149,60.093910,1.214481
23,34.537971,52.768269,12.165409,0.013427,0.045925,3.0,0.000732,2.427212e-02,33.0,21.0,0.027735,-0.149533,0.040608,0.000000,0.0,75.451454,0.649225,0.861515,0.616603,0.272165,0.200463,0.055002,0.000000,0.174173,0.006913,0.756154,0.621901,0.025099,0.535380,0.000000,0.230544,64.933678,1.161977
24,34.537971,67.908142,9.261819,0.017378,0.033213,2.0,0.000875,3.674139e-03,70.0,77.0,0.033873,-0.229690,0.038855,0.025398,0.0,75.451454,0.981718,1.024301,0.904198,0.590209,0.078184,0.078184,0.023915,0.059919,0.034098,0.851174,0.212026,0.081395,0.195841,0.083890,0.136387,77.169960,0.977731
27,9.288201,98.426880,8.117278,0.015359,0.016453,1.0,0.000000,0.000000e+00,67.0,63.0,0.024752,0.069475,0.027962,0.000000,0.0,111.682686,1.000000,1.036042,1.004620,0.772702,0.081282,0.015572,0.033359,0.024006,0.081282,0.944981,0.053170,0.125198,0.030984,0.081812,0.082470,106.544159,1.048229
28,9.288201,98.425285,6.405208,0.023443,0.028715,1.0,0.000000,0.000000e+00,34.0,39.0,0.028889,-0.015194,0.037591,0.013782,0.0,111.682686,1.000000,1.015834,0.957223,0.537257,0.243954,0.021863,0.056980,0.243954,0.006571,0.957306,0.360730,0.023169,0.102085,0.109104,0.065077,104.830490,1.065364


In [189]:
print 'Write this final endcap dataframe to disk'

print 'Processing feather'
feather.write_feather(final_endcap_df, 'final_endcap_df.feather')

print 'Processing parquet'
final_endcap_df.to_parquet('final_endcap_df.parquet', engine='fastparquet', compression='gzip')

print 'All done!'

Write this final endcap dataframe to disk
Processing feather
Processing parquet
All done!


In [188]:
usage()

12723.203125

In [190]:
mydf.shape

(14985167, 62)

In [191]:
del mydf


In [199]:
usage()

5851.421875

In [200]:
final_endcap_df.shape

(6380367, 33)

In [201]:
del endcap_df
del final_endcap_df

In [202]:
usage()

4365.05078125

In [204]:
del barrel_vars, endcap_vars


4365.09375

In [205]:
usage()

4365.09375

In [206]:
whos

Variable         Type        Data/Info
--------------------------------------
feather          module      <module 'pyarrow.feather'<...>ges/pyarrow/feather.pyc'>
garbageCollect   function    <function garbageCollect at 0x7ff13170b320>
gc               module      <module 'gc' (built-in)>
name             str         _ih
np               module      <module 'numpy' from '/cv<...>ages/numpy/__init__.pyc'>
os               module      <module 'os' from '/cvmfs<...>pt/lib/python2.7/os.pyc'>
pd               module      <module 'pandas' from '/c<...>ges/pandas/__init__.pyc'>
psutil           module      <module 'psutil' from '/c<...>ges/psutil/__init__.pyc'>
usage            function    <function usage at 0x7ff13170b1b8>


In [207]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [209]:
whos


Interactive namespace is empty.


In [210]:
for name in vars().keys():
    print(name)

_i
_sh
_dh
_i209
_i208
__builtins__
exit
_i210
__name__
__builtin__
quit
_iii
_oh
Out
get_ipython
In
_ii
_ih
